# Análise de Drug Discovery

Dados fornecidos pela alura, baseados nas bases da seguinte competição do Kaggle: kaggle.com/c/lish-moa/overview

Temos duas bases de dados,:

- Com as *features* dos tratamentos, como variáveis dos tratamentos (dose, tempo de exposição, composto utilizado) e consequências a nível de expressão gênica (colunas 'g-') e de viabilidade celular (colunas 'c-')
- Com os resultados em mecanismos de ação (MoA, na sigla em inglês)
- mskdlmasd

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
# importando bases

exp = pd.read_csv('../Dados/dados_experimentos.zip', compression='zip')
res = pd.read_csv('../Dados/dados_resultados.csv')

In [ ]:
exp.head()

,id,tratamento,tempo,dose,droga,g-0,g-1,g-2,g-3,g-4,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,-0.2479,-0.6208,-0.1944,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,0.2991,0.0604,1.0190,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,1.5540,-0.0764,-0.0323,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,-0.2656,0.5288,4.0620,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,0.9700,0.6919,1.4180,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [ ]:
exp.shape

(23814, 877)

In [ ]:
res.head()

,id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
res.shape

(23814, 207)

In [ ]:
# balanceamento por tratamento

exp['tratamento'].value_counts(normalize=True)

com_droga       0.921643
com_controle    0.078357
Name: tratamento, dtype: float64

In [ ]:
# balanceamento por droga

qtde_droga = exp['droga'].value_counts(normalize=True)
qtde_droga

cacb2b860    0.078357
87d714366    0.030150
9f80f3f77    0.010330
8b87a7a83    0.008524
5628cb3ee    0.008482
               ...   
ba08ac4bc    0.000042
e6b7b845f    0.000042
6e418401d    0.000042
81a463c0a    0.000042
125db1da8    0.000042
Name: droga, Length: 3289, dtype: float64

In [ ]:
exp['droga'].value_counts(normalize=True).describe()

count    3289.000000
mean        0.000304
std         0.001508
min         0.000042
25%         0.000252
50%         0.000252
75%         0.000252
max         0.078357
Name: droga, dtype: float64

In [ ]:
qtde_droga[qtde_droga>=exp['droga'].value_counts(normalize=True).describe()['50%']].count()/qtde_droga.count()

0.9382791121921557

In [ ]:
1/3289

0.0003040437823046519

In [ ]:
'jklas_mkasd_mk'.split(sep='_')[-1]

'mk'

In [ ]:
res2 = res.copy()
res2.columns = [col.split(sep='_')[-1] for col in res2.columns]

In [ ]:
pd.Series([col.split(sep='_')[-1] for col in res2.columns]).value_counts()

inhibitor            112
antagonist            32
agonist               28
activator              5
agent                  3
blocker                2
antimalarial           1
anticonvulsant         1
anti-inflammatory      1
local                  1
antiprotozoal          1
scavenger              1
diuretic               1
antiarrhythmic         1
immunosuppressant      1
stimulant              1
sensitizer             1
donor                  1
laxative               1
antihistamine          1
antifolate             1
medium                 1
analgesic              1
steroid                1
b                      1
antifungal             1
antioxidant            1
antiviral              1
secretagogue           1
id                     1
antibiotic             1
dtype: int64

In [ ]:
pivot = res2.melt(id_vars = 'id', var_name='ação').pivot_table(index='id', columns='ação', aggfunc='sum', margins=True)

In [ ]:
pivot.iloc[:,-1].value_counts()

1        12532
0         9367
2         1538
3          303
4           55
5           13
7            6
16844        1
Name: (value, All), dtype: int64

In [ ]:
filtro_col = [col for col in res.columns if col.split(sep='_')[-1]=='b' ]
filtro_col

['vitamin_b']